In [1]:
import sys
sys.path.append('..')
from db_init import db_get_df, db_save_df

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


tokenizer = AutoTokenizer.from_pretrained("UNIST-Eunchan/FLAN-T5-NLP-Paper-to-Question-Generation")
model = AutoModelForSeq2SeqLM.from_pretrained("UNIST-Eunchan/FLAN-T5-NLP-Paper-to-Question-Generation")


/Users/br/Projects/IT-Ptojekt-Chatbot/daibl/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# text = {
#     'abstract': "Hochschule mit vielen Professoren",
#     'introduction': "Die Hochschule hat viele Professoren, Professor Weber unterrichtet Grundlagen der Informatik, Professor Gallwitz unterrichtet Grundlagen der Medieninformatik, Professor Albrecht für Textanalytics"
# }
df = db_get_df("html_attrs")

In [9]:
df.loc[2,"text"]

'       Eine Pilotstudie. Oder: Wer kümmert sich um das Ehrenamt?  In der Praxis engagementfördernder Strukturen leisten „Verantwortliche für Engagement“ in Kommunen, Vereinen und Verbänden einen wichtigen Beitrag zum Gelingen des Engagements. Seit längerem ist dies u.a. ein Feld der Professionalisierung für Absolvent:innen des Studiengangs Soziale Arbeit. In der bisherigen Forschung wurde deren berufliches Setting jedoch kaum beleuchtet. Das Pilotprojekt wird hier erste valide Erkenntnisse bringen.    Laufzeit: 1. April 2022 bis 31. März 2023            Beteiligte         Projektleitung: Prof. Dr. Doris Rosenkranz (Fakultät SW) Projektmitarbeiter/in: Enya Buchner, M.A.           Zielsetzung         Verantwortliche für Engagement gibt es bundesweit in jedem Feld des Engagements. Doch bereits im „Wording“ für diese Funktionen zeigt sich eine große Heterogenität. Die These: Nach Organisation, Rahmenbedingungen oder Engagementfeld unterscheiden sich Bezeichnungen, berufliche Settings, Zug

In [4]:
summaries = []

In [5]:
prompt =  f""" 
Generate Question, Answer pair correspond to the following Website. 
{df.loc[3,'text']}
Question, Answer:
""".replace("\n", "")

inputs = tokenizer(prompt, max_length = 1024, truncation=True, padding="max_length", return_tensors="pt")
num_generate_sequence = 4 #8, 16, 2, 1
summary = model.generate(input_ids =inputs["input_ids"], max_new_tokens=100, do_sample = True, top_p = 0.95, num_return_sequences = num_generate_sequence)

In [6]:
summary

tensor([[    0,   571,    33,     8,  2996,  4468,    58,  6306,   134,  8569,
           908,   634,  2996,    33,  1860,    16,  1566,     5,    37,  2996,
            33,     3,   390,    30,  1895, 31972,  3819, 13416,     5,   328,
            33,  4760,    12,     8,   616,    13,   585,    11,  1659,  4632,
            16,     8,   616,    13,  1073,     5,     1],
        [    0,   363,    54,    25,   217,    81,     8,     3, 13777,   204,
          1195,    30,     8,   475,    58,  6306,   134,  8569,   908,   328,
           241,    12,  2519,  1038,  1707,    21,     8,  1656,    13,   481,
            11,  6040,    11,    12,  4797,     3,     9,  4388,  1229,    21,
             8,  1656,    13,     8,   481,     5,     1],
        [    0,  4073,   516,    19, 11264,    57,     8,  2968, 16682,  8231,
          1387,    58,  6306,   134,  8569,   908,     3, 13777,   204,     1,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             

In [7]:
decoded_summary = [tokenizer.decode(s, skip_special_tokens=False, clean_up_tokenization_spaces=True) for s in summary]
decoded_summary = [d.replace("<n>", " ").replace(tokenizer.pad_token, "").replace(tokenizer.eos_token, "") for d in decoded_summary]
summaries.append(decoded_summary)

In [8]:
decoded_summary

[' How are the courses conducted?[SEP]The courses are offered in English. The courses are based on existing Erasmus university partnerships. They are extended to the area of research and deepened in the area of education.',
 ' What can you see about the INT 2 projects on the website?[SEP] They want to promote internationalization for the benefit of students and faculty and to establish a worldwide network for the benefit of the students.',
 ' Which project is funded by the German Academic Exchange Service?[SEP] INT 2',
 ' To how long will the funding for this program?[SEP] The funding for this program is from the Federal Ministry of Education and Research since when was it started?']

In [ ]:
num_generate_sequence = 4 #8, 16, 2, 1
summaries = model.generate(input_ids =inputs["input_ids"], max_new_tokens=100, do_sample = True, top_p = 0.95, num_return_sequences = num_generate_sequence)


In [ ]:
decoded_summaries = [tokenizer.decode(s, skip_special_tokens=False, clean_up_tokenization_spaces=True) for s in summaries]
decoded_summaries = [d.replace("<n>", " ").replace(tokenizer.pad_token, "").replace(tokenizer.eos_token, "") for d in decoded_summaries]

In [ ]:
decoded_summaries

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text2text-generation", model="UNIST-Eunchan/FLAN-T5-NLP-Paper-to-Question-Generation")

In [ ]:
pipe(""" 
Generate Question, Answer pair correspond to the following research paper. 
[Abstract] In this work, we explore prompt tuning, a simple yet effective mechanism for learning soft prompts to condition frozen language models to perform specific downstream tasks. Unlike the discrete text prompts used by GPT-3, soft prompts are learned through backpropagation and can be tuned to incorporate signal from any number of labeled examples. Our end-to-end learned approach outperforms GPT-3's few-shot learning by a large margin. More remarkably, through ablations on model size using T5, we show that prompt tuning becomes more competitive with scale: as models exceed billions of parameters, our method closes the gap and matches the strong performance of model tuning (where all model weights are tuned). This finding is especially relevant in that large models are costly to share and serve, and the ability to reuse one frozen model for multiple downstream tasks can ease this burden. Our method can be seen as a simplification of the recently proposed prefix tuning of Li and Liang (2021), and we provide a comparison to this and other similar approaches. Finally, we show that conditioning a frozen model with soft prompts confers benefits in robustness to domain transfer, as compared to full model tuning. [Introduction] With the wide success of pre-trained large language models, a range of techniques has arisen to adapt these general-purpose models to downstream tasks. ELMo (Peters et al., 2018) proposed freezing the pre-trained model and learning a task-specific weighting of its per-layer representations. However, since GPT (Radford et al., 2018) and BERT (Devlin et al., 2019), the dominant adaptation technique has been model tuning (or fine-tuning), where all model parameters are tuned during adaptation, as proposed by Howard and Ruder (2018).More recently, Brown et al. (2020) showed that prompt design (or priming) is surprisingly effective at modulating a frozen GPT-3 model’s behavior through text prompts. Prompts are typically composed of a task description and/or several canonical examples. This return to freezing pre-trained models is appealing, especially as model size continues to increase. Rather than requiring a separate copy of the model for each downstream task, a single generalist model can simultaneously serve many different tasks. Unfortunately, prompt-based adaptation has several key drawbacks. Task description is error-prone and requires human involvement, and the effectiveness of a prompt is limited by how much conditioning text can fit into the model’s input. As a result, downstream task quality still lags far behind that of tuned models. For instance, GPT-3 175B fewshot performance on SuperGLUE is 17.5 points below fine-tuned T5-XXL (Raffel et al., 2020) (71.8 vs. 89.3) despite using 16 times more parameters. Several efforts to automate prompt design have been recently proposed. Shin et al. (2020) propose a search algorithm over the discrete space of words, guided by the downstream application training data. While this technique outperforms manual prompt design, there is still a gap relative to model tuning. Li and Liang (2021) propose prefix tuning and show strong results on generative tasks. This method freezes the model parameters and backpropagates the error during tuning to prefix activations prepended to each layer in the encoder stack, including the input layer. Hambardzumyan et al. (2021) simplify this recipe by restricting the trainable parameters to the input and output subnetworks of a masked language model, and show reasonable results on classifications tasks. In this paper, we propose prompt tuning as a further simplification for adapting language models. We freeze the entire pre-trained model and only allow an additional k tunable tokens per downstream task to be prepended to the input text. This soft prompt is trained end-to-end and can condense the signal from a full labeled dataset, allowing our method to outperform few-shot prompts and close the quality gap with model tuning (Figure 1). At the same time, since a single pre-trained model is recycled for all downstream tasks, we retain the efficient serving benefits of frozen models (Figure 2). While we developed our method concurrently with Li and Liang (2021) and Hambardzumyan et al. (2021), we are the first to show that prompt tuning alone (with no intermediate-layer prefixes or task-specific output layers) is sufficient to be competitive with model tuning. Through detailed experiments in sections 2–3, we demonstrate that language model capacity is a key ingredient for these approaches to succeed. As Figure 1 shows, prompt tuning becomes more competitive with scale. We compare with similar approaches in Section 4. Explicitly separating task-specific parameters from the generalist parameters needed for general language-understanding has a range of additional benefits. We show in Section 5 that by capturing the task definition in the prompt while keeping the generalist parameters fixed, we are able to achieve better resilience to domain shifts. In Section 6, we show that prompt ensembling, learning multiple prompts for the same task, can boost quality and is more efficient than classic model ensembling. Finally, in Section 7, we investigate the interpretability of our learned soft prompts. In sum, our key contributions are: 1. Proposing prompt tuning and showing its competitiveness with model tuning in the regime of large language models. 2. Ablating many design choices, and showing quality and robustness improve with scale. 3. Showing prompt tuning outperforms model tuning on domain shift problems. 4. Proposing prompt ensembling and showing its effectiveness. 
Question, Answer:
""".replace("\n", ""))